In [258]:
# %store -r final_dataframe

In [518]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from math import sqrt
from math import pi
from math import exp
import operator

In [519]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [520]:
def addTargetLabelToDataset(df):
    df['target'] = target_dataframe['target']
    return df

In [521]:
def seperate_by_target(df):
    dataset_by_target = list()
    for i in target_values:
        dataset_by_target.append(df.loc[df['target'] == i])
    return dataset_by_target

In [522]:
def calculate_stats(df):
    statistics_of_columns = []
    for column in df:
            new_d = [df[column].mean(),df[column].std(),len(df[column])]
            statistics_of_columns.append(new_d)
    del statistics_of_columns[-1]
    return statistics_of_columns

In [523]:
def calculate_probability(count, mean, stdev):
    x = target_values[count]
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [524]:
def get_target_counts_dict():
    target_count_dict = dict()
    c = 0
    for i in dataset_by_target:
        target_count_dict[str(target_values[c])] = len(i)
        c += 1
    return target_count_dict

In [536]:
# helper functions
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 # Calculate the standard deviation of a list of numbers
def stdev(numbers):
	avg = mean(numbers)
	variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
	return sqrt(variance)
def summarize_dataset(dataset):
	summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
	del(summaries[-1])
	return summaries
# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
	exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
	return (1 / (sqrt(2 * pi) * stdev)) * exponent
 
def calculate_class_probabilities(summaries, row):
	total_rows = sum([summaries[label][0][2] for label in summaries])
	probabilities = dict()
	for class_value, class_summaries in summaries.items():
		probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
		for i in range(len(class_summaries)):
			mean, stdev, _ = class_summaries[i]
			probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
	return probabilities


In [532]:
def coke(dataset_by_target,row):
    total_rows = len(df)
    probabilities = dict()
    # type(dataset_by_target[0])
    label_count = 0;
    for i in dataset_by_target:
        probabilities[i] = target_count_dict[str(i)] / float(total_rows)
        innerlist = dataset_by_target[str(i)]
        for j in range(len(innerlist)):
            mean, stdev, _ = innerlist[j]
            probabilities[i] *= calculate_probability(row[j],mean,stdev)
    return probabilities

In [533]:
def getMaxFromDictionaryValues(dictionary):
    return max(dictionary.items(), key=operator.itemgetter(1))[0]

In [526]:
os_path = '/home/dsp_team_gregory/gregory-p1/'
df = pd.read_csv (os_path+'data/X_train.txt.csv')
df.drop(df.columns[[0]], axis=1, inplace=True)

In [527]:
target_dataframe = pd.read_csv('https://storage.googleapis.com/uga-dsp/project1/files/y_train.txt', sep=" ", header=None)
target_dataframe.columns = ['target']

In [528]:
df = addTargetLabelToDataset(df)
# display(df.dtypes)
df_list = df.values.tolist()
for i in range(len(df_list)):
    r = len(df_list[i]) - 1
    df_list[i][len(df_list[i]) - 1] = int(df_list[i][len(df_list[i]) - 1])



In [529]:
target_values = df.target.unique()

In [530]:
dataset_by_target_dict = dict()
for i in target_values:
    temp = list()
    for j in range(len(df_list)):
        if(i == df_list[j][len(df_list[i]) - 1]):
            temp.append(df_list[j])
    dataset_by_target_dict[str(i)] = temp

In [531]:
summaries = dict()
for class_value, rows in dataset_by_target_dict.items():
    summaries[class_value] = summarize_dataset(rows)

innerlist = summaries['2']

In [535]:
probabilities = coke(summaries, df_list[0])
max_value = getMaxFromDictionaryValues(probabilities)
print(max_value)

1
